# S2.02 - Exploration algorithmique d'un problème

### BERHO Andoni
### BOURCIEZ Maxime
### TEXEIRA Jonathan
### TD II - TP 3

## Chemins piétons dans Bayonne





## Importation des données


In [7]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from math import pi,acos,asin,cos,sin

os.chdir("C:\\IUT\\Semestre 2\\S2.02 - Explo algorithmique d'un problème") # Pc Portable Maxime

# Importation des données
points = pd.read_csv("points.csv", encoding="ANSI", sep=";", index_col='id_point')
arcs = pd.read_csv("arcs.csv", encoding="ANSI", sep=";", index_col='id_arc')


# Récupération des listes sous forme de listes d'entiers

#### Dans cette boucle, nous allons aussi créer une colonne contenant le sommet de départ de l'arc, et celui d'arrivée. Ceci nous servira à trier les arcs pour garder uniquement ceux qui nous seront utiles, mais nous garderons les listes originelles afin de pouvoir calculer le poids exact.

In [9]:
# Ajout des nouvelles colonnes
arcs['listePoints'] = None
arcs['sommetDepart'] = None
arcs['sommetArrivee'] = None
arcs['numeroSommetDepart'] = None
arcs['numeroSommetArrivee'] = None

# Création des variables temporaires
tempoInt = ''
tempoString = 0
tempoList = []


# Transformation de la chaine de caractère en liste réelle,
# Puis sélection des sommets de départs etr d'arrivée dans tous les arcs
for ind in arcs.index:
    tempoList = []
    # Modifications de la chaîne de caractères
    tempoString = arcs.loc[ind]['lstpoints'].replace('[','')
    tempoString = tempoString.replace(']','')
    tempoString = tempoString.replace(' ','')
    tempoList = tempoString.split(',')
    
    arcs.at[ind,'listePoints'] = tempoList
    tempoList = []
    # Récupération des valeurs de la liste en entiers
    for pt in  arcs.at[ind,'listePoints'] :
        tempoList.append(int(pt))

    # Remplacer la liste de chaines par la liste d'entiers et remplir les autres colonnes
    arcs.at[ind,'listePoints'] = tempoList
    arcs.at[ind,'sommetDepart'] = int(arcs.loc[ind,'listePoints'][0])
    arcs.at[ind,'sommetArrivee'] = int(arcs.loc[ind,'listePoints'][-1])


# Changement de type
arcs = arcs.astype({'sommetDepart': 'int64', 'sommetArrivee': 'int64'})
  

## Mise en dataframe particulier pour les sommets que l'on garde

In [3]:
# Création de la liste des sommets distincts
listeSommets = []

for ind in arcs.index:    
    if arcs.loc[ind, 'sommetDepart'] not in listeSommets:
        listeSommets.append(arcs.loc[ind, 'sommetDepart'])
    if arcs.loc[ind, 'sommetArrivee'] not in listeSommets:
        listeSommets.append(arcs.loc[ind, 'sommetArrivee'])

# Import dans dataframe sommets
sommets = pd.DataFrame(listeSommets, columns=['Sommet'])

# Ajout d'un numéro par sommet et mise en index de cette variable
NB_SOMMETS = len(listeSommets)
numeroSommet = [i for i in range(NB_SOMMETS)]
sommets['numeroSommets'] = numeroSommet
sommets = sommets.set_index('Sommet')

In [4]:
#########################################################
# calcul de la distance entre deux points A et B dont  #
# on connait la lattitude et la longitude               #
#########################################################
def distanceGPS(latA,latB,longA,longB):
    # Conversions des latitudes en radians
    ltA=latA/180*pi
    ltB=latB/180*pi
    loA=longA/180*pi
    loB=longB/180*pi
    # Rayon de la terre en mètres 
    RT = 6378137
    # angle en radians entre les 2 points
    S = acos(round(sin(ltA)*sin(ltB) + cos(ltA)*cos(ltB)*cos(abs(loB-loA)),20)) # 4 poids à 0 si on arrondi à 14
    # distance entre les 2 points, comptée sur un arc de grand cercle
    return S*RT

In [5]:
# Matrice d'adjascense
matrice = [[0] * NB_SOMMETS for _ in range(NB_SOMMETS)]

# Ajout des numéros des sommets
for ind in arcs.index:
    arcs.loc[ind, 'numeroSommetDepart'] = sommets.loc[arcs.loc[ind, 'sommetDepart'], 'numeroSommets']
    arcs.loc[ind, 'numeroSommetArrivee'] = sommets.loc[arcs.loc[ind, 'sommetArrivee'], 'numeroSommets']

# Changement du type 
arcs = arcs.astype({'numeroSommetDepart': 'int64', 'numeroSommetArrivee': 'int64'})

# Génération de la matrice
for ind in arcs.index:
    matrice[arcs.loc[ind, 'numeroSommetDepart']][arcs.loc[ind, 'numeroSommetArrivee']] = 1
    matrice[arcs.loc[ind, 'numeroSommetArrivee']][arcs.loc[ind, 'numeroSommetDepart']] = 1



## Création de la matrice pondérée

In [10]:
# Ajout de lat et long à sommets
sommets['lat'] = points['lat']
sommets['lon'] = points['lon']

# Calculer le poid
for ind in arcs.index:
    poid = 0
    for i in range(len(arcs.loc[ind, 'listePoints']) - 1):
        latA = points.loc[arcs.loc[ind, 'listePoints'][i], 'lat']
        latB = points.loc[arcs.loc[ind, 'listePoints'][i + 1], 'lat']
        lonA = points.loc[arcs.loc[ind, 'listePoints'][i], 'lon']
        lonB = points.loc[arcs.loc[ind, 'listePoints'][i + 1], 'lon']
        poid = poid + distanceGPS(latA, latB, lonA, lonB)
    arcs.loc[ind, 'poid'] = poid

arcs = arcs.astype({'poid': 'float64'})

# Matrice pondérée
matricePonderee = [[0] * NB_SOMMETS for _ in range(NB_SOMMETS)]

for ind in arcs.index:
    matricePonderee[arcs.loc[ind, 'numeroSommetDepart']][arcs.loc[ind, 'numeroSommetArrivee']] = arcs.loc[ind, 'poid']
    matricePonderee[arcs.loc[ind, 'numeroSommetArrivee']][arcs.loc[ind, 'numeroSommetDepart']] = arcs.loc[ind, 'poid']


TypeError: list indices must be integers or slices, not NoneType